In [109]:
# code for autoreload of modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
import os
from src.utils import set_random_seed

In [111]:


DIR = r'C:\Users\u0148775\PycharmProjects\CreditScoring'
os.chdir(DIR)

set_random_seed(0)

In [112]:
from src.utils import load_config

# read dataconfig from CONFIG_DATA.yaml file:

dataconfig = load_config('config\CONFIG_DATA.yaml')
experimentconfig = load_config('config\CONFIG_EXPERIMENT.yaml')
methodconfig = load_config('config\CONFIG_METHOD.yaml')
evaluationconfig = load_config('config\CONFIG_EVALUATION.yaml')



In [113]:
from src.classes.experiment import Experiment

# Initialize experiment object
experiment = Experiment(dataconfig=dataconfig,
                        experimentconfig=experimentconfig,
                        methodconfig=methodconfig,
                        evaluationconfig=evaluationconfig)



Experiment created at:  2025-03-05-14-20
Task: lgd
Dataset: 00_lgd_toydata
CV splits:  2


In [114]:
experiment.run()

00_lgd_toydata loaded
00_lgd_toydata preprocessed
x shape:  (442, 10)
y shape:  (442,)


Cross-validation loop::   0%|          | 0/2 [00:00<?, ?it/s]

*Best hyperparameters (ab)* {'estimator': None, 'learning_rate': 0.01, 'loss': 'linear', 'n_estimators': 50, 'random_state': None}
*Best hyperparameters (ann)* {'activation': 'relu', 'batch_size': 256, 'dropout_rate': 0.1, 'early_stopping_delay': 5, 'epochs': 50, 'hidden_layer_size': 32, 'learning_rate': 0.001, 'num_hidden_layers': 1, 'random_seed': 0, 'weight_decay': 0.01}
*Best hyperparameters (cb)* {'iterations': 100, 'depth': 6, 'loss_function': 'RMSE', 'verbose': False}
*Best hyperparameters (dt)* {'ccp_alpha': 0.0, 'criterion': 'absolute_error', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}
*Best hyperparameters (en)* {'alpha': 0.5, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.1, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 't

Cross-validation loop::  50%|█████     | 1/2 [00:09<00:09,  9.14s/it]

*Best hyperparameters (xgb)* {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.01, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 5, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 50, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
*Best hyperparameters (ab)* {'estimator': None, 'learning_rate': 0.01, 'los

Cross-validation loop:: 100%|██████████| 2/2 [00:19<00:00,  9.62s/it]

*Best hyperparameters (xgb)* {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.01, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 3, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 50, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


In [115]:
experiment.results

{'ab': {0: {'mse': 4508.85574, 'mae': 52.2401, 'r2': 0.17566},
  1: {'mse': 3690.23805, 'mae': 49.9091, 'r2': 0.42206}},
 'ann': {0: {'mse': 28906.11753, 'mae': 153.09135, 'r2': -4.28479},
  1: {'mse': 28868.65715, 'mae': 149.94648, 'r2': -3.52122}},
 'cb': {0: {'mse': 3877.10048, 'mae': 49.43362, 'r2': 0.29117},
  1: {'mse': 3909.16342, 'mae': 51.32465, 'r2': 0.38777}},
 'dt': {0: {'mse': 8032.01018, 'mae': 71.46833, 'r2': -0.46846},
  1: {'mse': 6606.68326, 'mae': 64.42081, 'r2': -0.0347}},
 'en': {0: {'mse': 5399.57205, 'mae': 62.06747, 'r2': 0.01282},
  1: {'mse': 6321.24265, 'mae': 68.47216, 'r2': 0.01001}},
 'knn': {0: {'mse': 4195.62494, 'mae': 49.67119, 'r2': 0.23293},
  1: {'mse': 4374.62795, 'mae': 52.67421, 'r2': 0.31487}},
 'lgbm': {0: {'mse': 4194.23391, 'mae': 53.78811, 'r2': 0.23319},
  1: {'mse': 4955.82956, 'mae': 60.72692, 'r2': 0.22385}},
 'lr': {0: {'mse': 3093.66378, 'mae': 44.83555, 'r2': 0.4344},
  1: {'mse': 3098.71399, 'mae': 46.36557, 'r2': 0.5147}},
 'rf': {0